In [0]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, TimestampType

print("Iniciando a preparação e validação da estrutura da Camada Gold...")

# --- ETAPA 1: DEFINIÇÕES ---
catalog_name = "workspace"
gold_schema_name = "gold_db"
full_gold_schema = f"{catalog_name}.{gold_schema_name}"

# Lista das tabelas de dimensão que precisam ter as colunas SCD2
dimensions_to_check = ["dim_companies", "dim_locations", "dim_skills"] 

# Nome da tabela de checkpoint para as tabelas fato
checkpoint_table_name = "fact_load_checkpoints"

erros_prep = []

# --- ETAPA 2: GARANTIR QUE O SCHEMA GOLD EXISTA ---
try:
    print(f"Garantindo que o schema '{full_gold_schema}' exista...")
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {full_gold_schema}")
    spark.sql(f"USE {full_gold_schema}")
except Exception as e:
    msg = f"ERRO CRÍTICO ao criar/usar schema: {e}"
    print(msg)
    erros_prep.append(msg)

# --- ETAPA 3: VERIFICAR E ADICIONAR COLUNAS SCD2 ÀS DIMENSÕES ---
print("\nVerificando a estrutura das tabelas de dimensão para SCD Tipo 2...")

for table_name in dimensions_to_check:
    try:
        # Pega o schema atual da tabela
        current_schema = spark.read.table(table_name).schema
        existing_columns = [field.name.lower() for field in current_schema.fields]
        
        # Colunas que queremos garantir que existam
        columns_to_add = {
            "is_current": "BOOLEAN",
            "start_date": "TIMESTAMP",
            "end_date": "TIMESTAMP"
        }
        
        # String para o comando ALTER TABLE
        add_cols_sql_list = []
        
        for col_name, col_type in columns_to_add.items():
            if col_name.lower() not in existing_columns:
                print(f"Coluna '{col_name}' não encontrada na tabela '{table_name}'. Adicionando...")
                add_cols_sql_list.append(f"{col_name} {col_type}")
        
        if add_cols_sql_list:
            sql_command = f"ALTER TABLE {table_name} ADD COLUMNS ({', '.join(add_cols_sql_list)})"
            print(f"Executando: {sql_command}")
            spark.sql(sql_command)
            print(f"Tabela '{table_name}' atualizada com sucesso.")
        else:
            print(f"Tabela '{table_name}' já possui a estrutura SCD2 correta.")

    except Exception as e:
        # Se a tabela não existir, o spark.read.table vai falhar.
        if "TABLE_OR_VIEW_NOT_FOUND" in str(e):
            print(f"Aviso: Tabela de dimensão '{table_name}' ainda não existe. Será criada na carga da camada Gold.")
        else:
            msg = f"ERRO CRÍTICO na estrutura da tabela '{table_name}': {e}"
            print(msg)
            erros_prep.append(msg)


# --- ETAPA 4: GARANTIR QUE A TABELA DE CHECKPOINT EXISTA ---
print("\nVerificando a existência da tabela de checkpoint...")

try:
    # Tenta descrever a tabela para ver se ela existe
    spark.sql(f"DESCRIBE TABLE {checkpoint_table_name}")
    print(f"Tabela de checkpoint '{checkpoint_table_name}' já existe.")
except Exception as e:
    if "TABLE_OR_VIEW_NOT_FOUND" in str(e):
        try:
            print(f"Tabela de checkpoint '{checkpoint_table_name}' não encontrada. Criando agora...")
            checkpoint_schema = StructType([
                StructField("table_name", StringType(), True),
                StructField("last_processed_timestamp", TimestampType(), True)
            ])
            empty_df = spark.createDataFrame([], schema=checkpoint_schema)
            empty_df.write.format("delta").saveAsTable(checkpoint_table_name)
            print(f"Tabela de checkpoint '{checkpoint_table_name}' criada com sucesso.")
        except Exception as e_create:
            msg = f"ERRO CRÍTICO ao criar tabela de checkpoint: {e_create}"
            print(msg)
            erros_prep.append(msg)
    else:
        msg = f"ERRO CRÍTICO ao verificar checkpoint: {e}"
        print(msg)
        erros_prep.append(msg)

print("\n---------------------------------------------------")
if len(erros_prep) > 0:
    print(f"A preparação Gold terminou com {len(erros_prep)} erros:")
    for erro in erros_prep:
        print(erro)
    print("---------------------------------------------------")
    
    raise Exception("Falha na Preparação/Estrutura da Camada Gold. Verifique os logs.")
else:
    print("Preparação da Camada Gold finalizada com SUCESSO. Estruturas validadas.")